In [12]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input,decode_predictions
import numpy as np 
import pandas as pd

image_size = (224, 224)

model_mobileNet =  tf.keras.applications.MobileNet(
    input_shape=image_size + (3,),
    alpha=1.0,
    depth_multiplier=1,
    dropout=0.001,
    include_top=True,
    weights=None,
    input_tensor=None,
    pooling=None,
    classes=1,
    classifier_activation="sigmoid",
)
model_mobileNet_with_GAN =  tf.keras.applications.MobileNet(
    input_shape=image_size + (3,),
    alpha=1.0,
    depth_multiplier=1,
    dropout=0.001,
    include_top=True,
    weights=None,
    input_tensor=None,
    pooling=None,
    classes=1,
    classifier_activation="sigmoid",
)
#test_file_path = {'11.jpg','1.jpg'}#массив путей до тестовых изображений
model_list = {'vgg16' : model_vgg16,
              'resnet50': model_resnet50,
              'mobileNet':model_mobileNet}
def get_class(predictions):
    return decode_predictions(predictions,top = 1)[0][0][1]
def get_prob(predictions):
    return decode_predictions(predictions,top = 1)[0][0][2]

def img_to_arr(img):
    img = keras.preprocessing.image.load_img(
        file_path, target_size=(224,224))
    img_array = image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)
    img_array = preprocess_input(img_array)
    return img_array
def predict_(model, filepath):
    img = keras.preprocessing.image.load_img(file_path, target_size=(224,224))
    img_array = image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)
    img_array = preprocess_input(img_array)
    preds = model_list.get(model).predict(arr)
    print('Модель:{} Класс: {}, вероятность: {}\n'.format(model,get_class(preds),get_prob(preds)))
    return preds 
#тестирование сетей
# for model in model_list.keys():
#     for img in test_file_path:
#         predict_(model,img)

#добавить датасет +предобработать

#добавить список генераторов шумов
#добавить генеративно состязательную сеть
#добавить стандартные методы аугментации

In [2]:

def adv_acc(model, clear_image_path , dirty_image_path):
    TP,TN,FT,FN = 0,0,0,0
    predict_clear_cat = get_class(predict_(model,list_of_clear_cat))
    predict_dirty_cat = get_class(predict_(model,list_of_dirty_cat))
    
    predict_clear_dog = get_class(predict_(model,list_of_clear_dog))
    predict_dirty_dog = get_class(predict_(model,list_of_dirty_dog))
    
    if predict_cleat_cat =='cat' and predict_dirty_cat =='cat' or predict_clear_dog =='dog' and predict_dirty_dog =='dog':
        TP+=1
    if predict_cleat_cat !='cat' and predict_dirty_cat !='cat' or predict_clear_dog !='dog' and predict_dirty_dog !='dog':
        TN+=1
    if predict_cleat_cat =='cat' and predict_dirty_cat !='cat' or predict_clear_dog =='dog' and predict_dirty_dog !='dog':
        FN+=1
    if predict_cleat_cat !='cat' and predict_dirty_cat =='cat' or predict_clear_dog !='dog' and predict_dirty_dog =='dog':
        FP+=1    
    atr_acc = (TP + TN)/(TP + TN + FN + FP)
    return atr_acc


def diff_pred(model, clear_image_path , dirty_image_path):
    summ = 0
    for img,img_d in list_of_clear_data_cat,list_of_dirty_data_cat:
        predict_clear_cat = get_prob(predict_(model,list_of_clear_cat))
        predict_dirty_cat = get_prob(predict_(model,list_of_dirty_cat))
        summ += predict_clear_cat - predict_dirty_cat
    for img,img_d in list_of_clear_data_dog,list_of_dirty_data_dog:
        predict_clear_dog = get_prob(predict_(model,list_of_clear_dog))
        predict_dirty_dog = get_prob(predict_(model,list_of_dirty_dog))
        summ += predict_clear_dog - predict_dirty_dog
    return summ/(len(list_of_clear_data_cat) + len(list_of_clear_data_dog))


In [3]:
import os

num_skipped = 0
for folder_name in ("Cat", "Dog"):
    folder_path = os.path.join("PetImages", folder_name)
    for fname in os.listdir(folder_path):
        fpath = os.path.join(folder_path, fname)
        try:
            fobj = open(fpath, "rb")
            is_jfif = tf.compat.as_bytes("JFIF") in fobj.peek(10)
        finally:
            fobj.close()

        if not is_jfif:
            num_skipped += 1
            # Delete corrupted image
            os.remove(fpath)

print("Deleted %d images" % num_skipped)

batch_size = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "PetImages",
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "PetImages",
    validation_split=0.2,
    subset="validation",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)
data_augmentation = keras.Sequential(
    [
         layers.RandomFlip("horizontal"),
         layers.RandomRotation(0.1),
    ]
)
inputs = keras.Input(shape=image_size + (3,))
x = data_augmentation(inputs)
x = layers.Rescaling(1./255)(x)

train_ds = train_ds.prefetch(buffer_size=32)
val_ds = val_ds.prefetch(buffer_size=32)


Deleted 0 images
Found 23422 files belonging to 2 classes.
Using 18738 files for training.
Found 23422 files belonging to 2 classes.
Using 4684 files for validation.


In [8]:
# model_vgg16 = keras.models.load_model('vgg16_save_at_11.h5')
# model_vgg16.compile(
#     optimizer=keras.optimizers.Adam(1e-3),
#     loss="binary_crossentropy",
#     metrics=["accuracy"],
# )
# callbacks_for_vgg16 = [
#     keras.callbacks.ModelCheckpoint("2_vgg16_save_at_{epoch}.h5"),
# ]
# model_vgg16.fit(
#     train_ds, epochs=39, callbacks = callbacks_for_vgg16, validation_data=val_ds,
# )
# model_resnet50.compile(
#     optimizer=keras.optimizers.Adam(1e-3),
#     loss="binary_crossentropy",
#     metrics=["accuracy"],
# )


# callbacks_for_resnet50 = [
#     keras.callbacks.ModelCheckpoint("resnet50_save_at_{epoch}.h5"),
# ]

# model_resnet50.fit(
#     train_ds, epochs=50, callbacks = callbacks_for_resnet50, validation_data=val_ds,
# )

In [6]:

# модель MobileNet обучена на нормальных данных
model_mobileNet.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)
callbacks_for_mobilenet = [
    keras.callbacks.ModelCheckpoint("mobilenet_save_at_{epoch}.h5"),
]

model_mobileNet.fit(
    train_ds, epochs=50, callbacks = callbacks_for_mobilenet, validation_data=val_ds,
)
#дальше зашумить картинки - предикт всех картинок(и нормальных и зашумленных и пропуск через функции метрик)
#дальше обучить на датасете с грязными данными и повторить прогон метрик
#дальше применить встроенные функции аугментации и повторить прогон метрик


Epoch 1/50
586/586 [==============================] - 1405s 2s/step - loss: 0.6476 - accuracy: 0.6298 - val_loss: 0.7014 - val_accuracy: 0.5009
Epoch 2/50
586/586 [==============================] - 1367s 2s/step - loss: 0.5319 - accuracy: 0.7309 - val_loss: 0.6580 - val_accuracy: 0.7306
Epoch 3/50
586/586 [==============================] - 1398s 2s/step - loss: 0.4509 - accuracy: 0.7883 - val_loss: 0.6449 - val_accuracy: 0.7237
Epoch 4/50
586/586 [==============================] - 1360s 2s/step - loss: 0.3806 - accuracy: 0.8265 - val_loss: 0.8186 - val_accuracy: 0.7385
Epoch 5/50
586/586 [==============================] - 1347s 2s/step - loss: 0.3174 - accuracy: 0.8610 - val_loss: 0.5366 - val_accuracy: 0.7908
Epoch 6/50
586/586 [==============================] - 1497s 3s/step - loss: 0.2654 - accuracy: 0.8872 - val_loss: 0.4259 - val_accuracy: 0.8474
Epoch 7/50
586/586 [==============================] - 1385s 2s/step - loss: 0.2296 - accuracy: 0.9043 - val_loss: 0.4888 - val_accuracy:

In [ ]:
# list_of_clear_data_cat = os.listdir('PetImages/Cat')
# list_of_clear_data_dog = os.listdir('PetImages/Dog')
# list_of_dirty_data_cat = os.listdir('PetImages_dirty/Cat')
# list_of_dirty_data_dog = os.listdir('PetImages_dirty/Dog')

In [ ]:
# loss_object = tf.keras.losses.CategoricalCrossentropy()
# #Тестировать только после обучения модели.
# def create_adversarial_pattern(input_image, #изображение которое предсказываем
#                                input_label, #ожидаемый класс
#                                model):#модель НС
#     with tf.GradientTape() as tape:
#         tape.watch(input_image)
#         prediction = predict_(model,input_label)
#         loss = loss_object(predict_(, prediction)

#     # Get the gradients of the loss w.r.t to the input image.
#     gradient = tape.gradient(loss, input_image)
#     # Get the sign of the gradients to create the perturbation
#     signed_grad = tf.sign(gradient)
#     return signed_grad
# #массив предсказаний класса - input_label



In [ ]:
# for img in list_of_clear_data_cat:
#     perturbations = create_adversarial_pattern(img, get_class())
#     new_img = open("PetImages_dirty/Cat" + img,"w")
#     new_img.write('PetImages/Cat'+ img + perturbations)
#     new_img.close()

In [64]:
train_ds_with_gan = tf.keras.preprocessing.image_dataset_from_directory(
    "PG",
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)
val_ds_with_gan = tf.keras.preprocessing.image_dataset_from_directory(
    "PG",
    validation_split=0.2,
    subset="validation",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

Found 33748 files belonging to 2 classes.
Using 26999 files for training.
Found 33748 files belonging to 2 classes.
Using 6749 files for validation.


In [65]:
#модель MobileNet обучена на выборке+сгенерированные примеры
#model_mobileNet_with_GAN = keras.models.load_model('mobilenet_save_at_50.h5')
#на случай нежелательных результатов - дообучить основную модель ТОЛЬКО на сгенерированных данных
#model_mobileNet_with_GAN = keras.models.load_model('mobilenet_with_gan_save_at_1.h5')
model_mobileNet_with_GAN.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)
callbacks_for_mobilenet_with_gan = [
    keras.callbacks.ModelCheckpoint("mobilenet_with_gan_save_at_{epoch}.h5"),
]
model_mobileNet_with_GAN.fit(
    train_ds_with_gan, epochs=50, callbacks=callbacks_for_mobilenet_with_gan, validation_data=val_ds_with_gan,
)


Epoch 1/50
844/844 [==============================] - 4127s 5s/step - loss: 0.6006 - accuracy: 0.6680 - val_loss: 0.5420 - val_accuracy: 0.7274
Epoch 2/50
844/844 [==============================] - 2030s 2s/step - loss: 0.4200 - accuracy: 0.8012 - val_loss: 0.4164 - val_accuracy: 0.8124
Epoch 3/50
844/844 [==============================] - 2040s 2s/step - loss: 0.3018 - accuracy: 0.8656 - val_loss: 0.2661 - val_accuracy: 0.8840
Epoch 4/50
844/844 [==============================] - 2035s 2s/step - loss: 0.2242 - accuracy: 0.9050 - val_loss: 0.5886 - val_accuracy: 0.8068
Epoch 5/50
844/844 [==============================] - 2033s 2s/step - loss: 0.1832 - accuracy: 0.9243 - val_loss: 0.3085 - val_accuracy: 0.8791
Epoch 6/50
844/844 [==============================] - 2035s 2s/step - loss: 0.1500 - accuracy: 0.9389 - val_loss: 0.2975 - val_accuracy: 0.8939
Epoch 7/50
844/844 [==============================] - 2001s 2s/step - loss: 0.1263 - accuracy: 0.9486 - val_loss: 0.3543 - val_accuracy:

In [66]:
batch_size = 32
train_ds_with_aug = tf.keras.preprocessing.image_dataset_from_directory(
    "PetImages",
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
    label_mode = 'binary'
)
val_ds_with_aug = tf.keras.preprocessing.image_dataset_from_directory(
    "PetImages",
    validation_split=0.2,
    subset="validation",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
    label_mode = 'binary'
)
train_ds_with_aug = train_ds_with_aug.prefetch(buffer_size=32)
val_ds_with_aug = val_ds_with_aug.prefetch(buffer_size=32)




Found 23422 files belonging to 2 classes.
Using 18738 files for training.
Found 23422 files belonging to 2 classes.
Using 4684 files for validation.


In [69]:
data_augmentation = keras.Sequential(
    [
         layers.RandomFlip("horizontal"),
         layers.RandomRotation(0.1),
        
    ]
)
inputs = keras.Input(shape=image_size + (3,))
x = data_augmentation(inputs)
x = layers.Rescaling(1./255)(x)

train_ds_with_aug = train_ds_with_aug.prefetch(buffer_size=32)
val_ds_with_aug = val_ds_with_aug.prefetch(buffer_size=32)
model_mobileNet_with_aug =  tf.keras.applications.MobileNet(
    input_shape=image_size + (3,),
    alpha=1.0,
    depth_multiplier=1,
    dropout=0.001,
    include_top=True,
    weights=None,
    input_tensor=x,
    pooling=None,
    classes=1,
    classifier_activation="sigmoid",
)

In [70]:
model_mobileNet_with_aug.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)
callbacks_for_mobilenet_with_aug = [
    keras.callbacks.ModelCheckpoint("mobilenet_with_aug_save_at_{epoch}.h5"),
]
model_mobileNet_with_aug.fit(
    train_ds_with_aug, epochs=50, callbacks=callbacks_for_mobilenet_with_aug, validation_data=val_ds_with_aug,
)

Epoch 1/50
586/586 [==============================] - 1429s 2s/step - loss: 0.6566 - accuracy: 0.6216 - val_loss: 1.0268 - val_accuracy: 0.4957
Epoch 2/50
586/586 [==============================] - 1448s 2s/step - loss: 0.5703 - accuracy: 0.6983 - val_loss: 0.6509 - val_accuracy: 0.6386
Epoch 3/50
586/586 [==============================] - 1475s 3s/step - loss: 0.5002 - accuracy: 0.7530 - val_loss: 0.6741 - val_accuracy: 0.6808
Epoch 4/50
586/586 [==============================] - 1486s 3s/step - loss: 0.4513 - accuracy: 0.7846 - val_loss: 0.4984 - val_accuracy: 0.7487
Epoch 5/50
586/586 [==============================] - 1484s 3s/step - loss: 0.4004 - accuracy: 0.8176 - val_loss: 0.5281 - val_accuracy: 0.7504
Epoch 6/50
586/586 [==============================] - 1494s 3s/step - loss: 0.3464 - accuracy: 0.8443 - val_loss: 0.3965 - val_accuracy: 0.8320
Epoch 7/50
586/586 [==============================] - 1439s 2s/step - loss: 0.2975 - accuracy: 0.8704 - val_loss: 0.4472 - val_accuracy: